In [ ]:
'''
Modules required for imaging and data analysis
'''

import pandas as pd
import numpy as np
import matplotlib##
import matplotlib.pyplot as plt##
import scipy.interpolate
%matplotlib inline
import os
import math
import seaborn as sns
import scipy
import skimage
from skimage import data
from skimage import io
import os
from skimage import filters
import math
from scipy.optimize import curve_fit
from scipy.stats.distributions import  t
from skimage.filters import threshold_otsu, threshold_adaptive
from skimage.morphology import remove_small_objects
import statsmodels.api as sm
import bottleneck as bn
nanmean = bn.nanmean

In [ ]:
'''
Create a DataFrame to convert frames in minutes

'''
total_number_of_TimeIDs = 150  #number of frames

time_step = 5  #frame rate in minutes

TimeIDs = range(1, total_number_of_TimeIDs + 1, 1) #+1 because the last number is not accounted
Time_mins = []

for i in TimeIDs:
    t = time_step*(i-1)
    Time_mins.append(t)
    
time_conversion = pd.DataFrame({"TimeID" : TimeIDs, "Time" : Time_mins})

time_conversion

In [ ]:
'''
Create a list with the names of the .tif files to be loaded and analyzed
'''

PATH_to_the_TIFs = "Z:/Experiments/TORC1/20190506_wf(new)_SFP1-TOD6-pHtdGFP_agar_mm_rep4/Processed_tiffs/"  #path to the folder containin the .tif files
all_files = os.listdir(PATH_to_the_TIFs)

tif_files = []
for f in all_files:
    if f.endswith(".tif"):
        tif_files.append(f)
        
tif_files

In [ ]:
'''
Creating a list of .csv files with the segmentation info created by BudJ

'''
PATH_to_the_TIFs2 = "Z:/Experiments/TORC1/20190506_wf(new)_SFP1-TOD6-pHtdGFP_agar_mm_rep4/Analysis/" # path to the folder containing the BudJ output files
files = []
for i in os.listdir(PATH_to_the_TIFs2):
    if ".csv" in i:
            prefix_position = "pos" + i[-22:-20]+"_"
            files.append((i, prefix_position))
            
files

In [ ]:
'''
Creating a DataFrame containing the segmenation info from BudJ
'''

initial_table = pd.DataFrame({})
for f in files:
    pos = pd.read_csv(f[0], header=0, index_col=0)
    pos["Cell_pos"] = f[1] + pos["Cell"].map(str)
    pos = pos.loc[:, ["TimeID", "Cell_pos", "volume", "x", "y", "major R", "minor r", "angle"]]
    initial_table = pd.concat([initial_table, pos])
    
initial_table = pd.merge(initial_table, time_conversion, on="TimeID")
initial_table

In [ ]:
'''
Number of individual cells segmented by BudJ
'''

individual_cells = sorted(list(set(initial_table["Cell_pos"])))
len(individual_cells)

In [ ]:
'''
Creates a dictionary containing the budding events (in frame) for each cell
'''

buddings_SFP1 = {"pos01_1" : [1, 34, 53, 74, 99, 119],
"pos01_2" : [9, 34, 58, 79, 102, 125],
..........
}

In [ ]:
'''
Creates a dictionary containing the karyokinesis events (in frame) for each cell
'''

kariokinesis_SFP1 = {"pos01_1" : [17, 48, 68, 92, 113],
"pos01_2" : [26, 50, 73, 94, 116],
......
}

In [ ]:
print(len(buddings_SFP1))
print(len(kariokinesis_SFP1))

In [ ]:
'''
Creates a function that converts the ellipse parameters from BudJ into pixels unit
'''

def ellipse(time_point, BudJ_table, scaling_factor):
    h = float(BudJ_table[BudJ_table["TimeID"] == time_point]['x'])/scaling_factor
    k = float(BudJ_table[BudJ_table["TimeID"] == time_point]['y'])/scaling_factor
    a = float(BudJ_table[BudJ_table["TimeID"] == time_point]["major R"])/scaling_factor
    b = float(BudJ_table[BudJ_table["TimeID"] == time_point]["minor r"])/scaling_factor
    A = float(BudJ_table[BudJ_table["TimeID"] == time_point]['angle'])*(math.pi/180)
    return h, k, a, b, A

In [ ]:
def round_up_to_odd(f):
    return int(np.ceil(f) // 2 * 2 + 1)

In [ ]:
### this code apply the mask defined by the BudJ segmentatio to each cell at each time point and measures 
### the N/C ratio for that cell at every time point

scaling_factor = 0.16 #microns per pixel, 100x objective

initial_table_recalculated_SFP1 = pd.DataFrame({})

c = 0
for pos in ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20']:  #list of all the XY position to be analyzed

    filename = os.path.join(skimage.data_dir, PATH_to_the_TIFs+"20190506_wf(new)_sfp1-tod6_agar_mm_gl_rep3_xy"+pos+".nd2.tif") 
    cell_tif = io.imread(filename) # load the raw images files 
    
    for cell in individual_cells:
        if 'pos'+pos in cell:
                temp = initial_table[initial_table["Cell_pos"] == cell] #extract the BudJ info for a particular cell
                temp = temp.sort_values(by="Time")
                
                time_axis = list(temp["TimeID"]) #extract time axis
                
                ##List of all the parameters to be measured for each cell at each time point
                GFP_cell = []
                GFP_cyto1 = []
                GFP_nucleus1 = []
                Ratio1 = []
                RFP_total = []
                              
                for t in time_axis:
                    
                        
                        t_in_tiff = int(t) - 1

                        h, k, a, b, A = ellipse(t, temp, scaling_factor) 

                        #creating the mask corresponding to the BudJ ellipse
                        image = cell_tif[t_in_tiff,:,:,2] #Extract one channel, at time point "t_in_tiff", does not matter which channel
                        nrows, ncols = image.shape
                        row, col = np.ogrid[:nrows, :ncols]
                        
                        ### it creates a mask corresponding to the ellipse fitted by BudJ to segment the cell at this particular frame
                        inner_disk_mask = ((((col-h-1)*math.cos(A)+(row-k-1)*math.sin(A))**2)/(a**2) + (((col-h-1)*math.sin(A)-(row-k-1)*math.cos(A))**2)/(b**2) - 1 < 0)
  
                        ###Calculate variables in the GFP channel like mean intensity, std,...
                        imageGFP = cell_tif[t_in_tiff,:,:,1] # extract GFP channel
                        imageGFP_corr = imageGFP[inner_disk_mask == True] # apply cell mask
                        mean_GFP = np.mean(imageGFP_corr) # calculate mean GFP intensity inside the cell mask
                        GFP_cell.append(mean_GFP)
                        GFP_std.append(np.std(imageGFP[inner_disk_mask == True])) # calculate the STD of the GFP intensity inside the cell mask

                        ### Nuclear mask
                        imageRFP = cell_tif[t_in_tiff,:,:,2] # extract the RFP channel
                        imageRFP_corr = imageRFP*inner_disk_mask # apply cells mask
                        thr_mask = imageRFP_corr > 10 # use a simple intensity thrshold to create a mask of the nucleus
                        
                        centroid1 = scipy.ndimage.measurements.center_of_mass((imageGFP*inner_disk_mask)*thr_mask) # calculate the centroid of the nuclear mask
                        centroid1 = np.nan_to_num(centroid1)
                        b1 = centroid1[1]
                        a1 = centroid1[0]+1
                        r = 3 # define radious for small nucelar mask, in pixels
                        r1 = 9 # define radious for big nucelar mask, in pixels
                            
                        x1,y1 = np.ogrid[-a1: nrows-a1, -b1: ncols-b1]
                        
                        disk_mask_nuc1 = x1*x1 +y1*y1 < r*r #create small, circular, nuclear mask 
                        disk_mask_cyto1 = x1*x1 +y1*y1 < r1*r1 #create big, circular, nuclear mask,
                        diff1 = np.logical_and(disk_mask_cyto1, inner_disk_mask) #overlap big nucelar mask with whole cell mask to avoid including pixels outside the whole cell mask
                        mask_of_cytoplasm1 = inner_disk_mask ^ diff1 #define the cytosoli mask by subtractin the big nuclear mask from the whole cell mask
                        
                        #RFP
                        RFP_total.append(np.sum(imageRFP_corr[thr_mask == True]))  #calculate total RFP signal insied the small nuclear mask
                    
                        #GFP in the nucleus
                        nucleus_mean1 = np.mean(imageGFP[disk_mask_nuc1 == True]) #calculate mean GFP signal inside the small nuclear mask, that is what we consider the nucelar GFP concentration
                        GFP_nucleus1.append(nucleus_mean1)
                    
                        #GFP in the cytosol
                        cyto_mean1 = np.mean(imageGFP[mask_of_cytoplasm1 == True]) #calculate mean GFP signal inside the cytosolic mask, that is what we consider the cytosolic GFP concentraion
                        GFP_cyto1.append(cyto_mean1)
                       
                        #Ratio
                        Ratio1.append((nucleus_mean1)/(cyto_mean1)) #calculate the N/C ratio
                       
                ###save calculated variables in DataFrame
                temp["GFP_cell"] = pd.Series(GFP_cell, index=temp.index)
                temp["GFP_std"] = pd.Series(GFP_std, index=temp.index)
                temp["GFP_nucleus1"] = pd.Series(GFP_nucleus1, index=temp.index)
                temp["GFP_cyto1"] = pd.Series(GFP_cyto1, index=temp.index)
                temp["Ratio1"] = pd.Series(Ratio1, index=temp.index)
                temp["RFP_conc"] = pd.Series(RFP_conc, index=temp.index)
                temp["RFP_total"] = pd.Series(RFP_total, index=temp.index)
                
                
             
                initial_table_recalculated_SFP1 = pd.concat([initial_table_recalculated_SFP1, temp])
                
                c += 1
                print cell,

In [ ]:
initial_table_recalculated_SFP1 = initial_table_recalculated_SFP1.reset_index(drop="true")

In [ ]:
initial_table_recalculated_SFP1[pd.isnull(initial_table_recalculated_SFP1).any(axis=1)]

In [ ]:
initial_table_recalculated_SFP1.to_excel("Sfp1_agar_rep1.xlsx")

In [ ]:
initial_table_recalculated_SFP1 = pd.read_excel("Sfp1_agar_rep1.xlsx")

In [ ]:
initial_table_SFP1 = initial_table_recalculated_SFP1

In [ ]:
## convert budding events in minutes

buddings_control_SFP1 = {}


for cell in individual_cells_SFP1:
        control = []
        
        for i in buddings_SFP1[cell]:
            budding_time = float(time_conversion[time_conversion["TimeID"] == i]["Time"])
            control.append(i)
                
        buddings_control_SFP1[cell] = control

In [ ]:
## convert karyokinesis events in minutes

kariokinesis_control_SFP1 = {}

for cell in individual_cells_SFP1:
        control = []
        
        for i in kariokinesis_SFP1[cell]:
            kariokinesis_time = float(time_conversion[time_conversion["TimeID"] == i]["Time"])
            control.append(i)
                
        kariokinesis_control_SFP1[cell] = control

In [ ]:
'''alignment karyokinesis to karyokinesis
   Create a list where for each cell the information regarding each cell cycle (karyokiensis event, budding event, next karyokinesis event
   number of point in between the first part and the second part of each cell cycle, duration of each part of each cell cycle) are stored'''

cell_cycles_list_SFP1 = []


buddings_group_SFP1 = buddings_control_SFP1
kariokinesis_group_SFP1 = kariokinesis_control_SFP1


for cell in buddings_group_SFP1:
    
    table = initial_table_SFP1[initial_table_SFP1["Cell_pos"] == cell] #extract all variables corresponding to a cell
    
    buddings_of_the_cell = buddings_group_SFP1[cell]
    kariokinesis_of_the_cell = kariokinesis_group_SFP1[cell]
    
    for i in range(len(kariokinesis_of_the_cell)):
        if i != len(kariokinesis_of_the_cell)-1:
            print cell
            
            start1 = float(time_conversion[time_conversion["TimeID"] == kariokinesis_of_the_cell[i]]["Time"])
            end1 = float(time_conversion[time_conversion["TimeID"] == buddings_of_the_cell[i+1]]["Time"])### KEEP the FLOAT function here
            start2 = float(time_conversion[time_conversion["TimeID"] == buddings_of_the_cell[i+1]]["Time"])
            end2 = float(time_conversion[time_conversion["TimeID"] == kariokinesis_of_the_cell[i+1]]["Time"])### KEEP the FLOAT function here
            
            
            duration1 = end1 - start1
            duration2 = end2 - start2
            
            time_points_for_aligning_at_bud1 = (1/duration1)*np.array(np.arange(start1-start1, end1-start1+1, 5))
            time_points_for_aligning_at_bud2 = (1/duration2)*np.array(np.arange(start2-start2, end2-start2+1, 5))

            cell_cycles_list_SFP1.append((cell, start1, end1, start2, end2, time_points_for_aligning_at_bud1, time_points_for_aligning_at_bud2, duration1, duration2))

In [ ]:
### plot the distributions of the cell cycle phases duration
%matplotlib inline
import matplotlib.pyplot as plt
sg2m = []
g1 = []
tot = []
plt.figure(1,(9,9))
for cycles in cell_cycles_list_SFP1: 
        if (cycles[7]+cycles[8]) < 200: # exclude cell cycles longer than 200 minutes
            dur1 = cycles[7] #G1 duration
            dur2 = cycles[8] #S/G2/M duration
            dur3 = cycles[7] + cycles[8]
            sg2m.append(dur2)
            g1.append(dur1)
            tot.append(dur3)
    
plt.hist(tot,15,label= 'Full cell cycle duration')
plt.hist(g1,15,label = 'G1 duration')
plt.hist(sg2m,15,label = 'S/G2/M duration')
plt.xlabel("min")
plt.legend()
plt.xlim(0, 500)
plt.savefig("SFP1_cell_cycles_hist_k2k.png")

In [ ]:
print(np.mean(tot))
print(np.mean(g1))
print(np.mean(sg2m))

In [ ]:
#### in this code each variable for each cell is splitted according to the cell cycle events and each part is then interpolated
#### with a fixed number of points defined by the average duartion of G1 and S/G2/M and their ratio, the sum have to be 80.

variables = ["volume","GFP_cell","GFP_nucleus1","GFP_cyto1","Ratio1","GFP_std","RFP_total"] # variables to be aligned and interpolated

L = len(variables)
k=0

c = 1

all_time_points1 = np.linspace(0, 1, 80) ## total number of points used for the interpolation of each cell cycle series
all_time_points2 = np.linspace(0, 1, 27) ## numnber of points used to interpolate the first part (G1) of each cell cycle series,
all_time_points3 = np.linspace(0, 1, 54) ## numnber of points used to interpolate the second part (SG2M) of each cell cycle series
                                         ## +1 point that will be exclude since it is already contained in the fisrt part (G1) of the cell cycle

cc_df1_SFP1 = pd.DataFrame({})
cc_df_small1_SFP1 = pd.DataFrame({})

cell_cycle = 0

for cycle in cell_cycles_list_SFP1:
                
                pos_cell = cycle[0]
                start1 = cycle[1]
                end1 = cycle[2]
                start2 = cycle[3]
                end2 = cycle[4]
                time1 = cycle[5]
                time2 = cycle[6]
                new_time1 = all_time_points1
                new_time2 = all_time_points2
                new_time3 = all_time_points3
                
                cell_cycle += 1
                print pos_cell
                
                for i in range(L):
                    variable = variables[i]                    
                    big_data_table = initial_table_SFP1                  
                    table = big_data_table[big_data_table["Cell_pos"] == pos_cell] # extract the variables corresponding to a cell
            
                    sensor1 = table[(table["Time"] >= start1) & (table["Time"] <= end1)][variable] #split a variable from karyokinesis to budding
                    sensor2 = table[(table["Time"] >= start2) & (table["Time"] <= end2)][variable] #split a variable from budding to karyokinesis
                   
                    if len(sensor1) != len(time1):
                
                        continue
                    
                    f1 = scipy.interpolate.interp1d(time1, sensor1) #linear interpolation
                    f2 = scipy.interpolate.interp1d(time2, sensor2) #linear interpolation 
                   
                    new_sensor1 = f1(new_time2)
                    new_sensor2 = f2(new_time3)
                    
                    cc_df_small1_SFP1["TimeID"] = new_time1
                    cc_df_small1_SFP1[variable] = np.append(new_sensor1, new_sensor2[1:]) 
                    
                cc_df_small1_SFP1["Cell_cycle"] = cell_cycle                        
                cc_df1_SFP1 = pd.concat([cc_df1_SFP1, cc_df_small1_SFP1])